In [16]:
import pandas as pd
import numpy as np
import datetime
import re
import os

In [17]:
# create a few statistic df
stat_columns = ['entity_name', 'batch_num', 'error_type']
df_valid = pd.DataFrame(columns=stat_columns)
df_Hrat = pd.DataFrame(columns=stat_columns)
df_Amnt = pd.DataFrame(columns=stat_columns)
df_Gend = pd.DataFrame(columns=stat_columns)

In [28]:
def validate_batch(batch_file):
    
    DTYPE={
    0:  str,      # RecNum                ## str in original schema
    1:  int,      # FiscalYear            ## str in original schema
    2:  str,      # entity name
    3:  str,      # Fund1
    4:  str,      # Fund2
    5:  str,      # Fund3
    6:  str,      # Fund4
    7:  str,      # Org1
    8:  str,      # Org2
    9:  str,      # Org3
    10: str,      # Org4
    11: str,      # Org5
    12: str,      # Org6
    13: str,      # Org7
    14: str,      # Org8
    15: str,      # Org9
    16: str,      # Org10
    17: str,      # Trans Type
    18: str,      # Cat1                  ## conditional
    19: str,      # Cat2
    20: str,      # Cat3
    21: str,      # Cat4
    22: str,      # Cat5
    23: str,      # Cat6
    24: str,      # Cat7
    25: str,      # PCV name              ## conditional
    26: str,      # DBA
    27: str,      # vndr Id
#   28: str,      # posting date
    29: str,      # Trans Desc
    30: str,      # transaction ID
    31: str,      # ref id
    32: str,      # contact name
    33: str,      # contract num
    34: str,      # title
    35: str, # hourly rate           ## float take NAs
    36: str,      # gender
    37: str, # amount
    38: str,      # masked
    39: str,      # UCA
            }


    #path_str = "H:/Transparency/Transparent Utah 3.0/Raw Batches/rawBatch/_data_56125.txt"
    #path_str = "H:/Transparency/Transparent Utah 3.0/Raw Batches/rawBatch/" + batch_file
    path_str = batch_file
    #batch_num =re.search("\d{5}(?=.txt$)",path_str).group()
    batch_num =re.search("(?<=\/)(\s|\w)*(?=.txt$)",path_str).group()
    batch_name  = 'batch_'+batch_num 



    df=pd.read_csv(path_str,
                   header=None, sep='|',
                   thousands=',',
                   low_memory=False,
                   parse_dates=[28],
                   encoding='cp1252',
                   dtype=DTYPE
                  )




    info = [
        'RcNm', 'FsYr', 'Name',
        'Fnd1', 'Fnd2', 'Fnd3', 'Fnd4',
        'Org1', 'Org2', 'Org3', 'Org4', 'Org5', 'Org6', 'Org7', 'Org8', 'Org9', 'OrgX',
        'Type',
        'Cat1', 'Cat2', 'Cat3', 'Cat4', 'Cat5', 'Cat6', 'Cat7',
        'Vndr', 'VDBA', 'VIDC', 'Pdat', 'Desc', 'TID', 'TRef', 'Cnam', 'Cnum', 'PTit', 'Hrat', 'Gend', 'Amnt', 'PI',
    ]
    uca = [ 'UCA', ]
    psd = [
        'Prg1', 'Prg2', 'Prg3', 'Prg4', 'Prg5', 'Prg6', 'Prg7',
        'Fnc1', 'Fnc2', 'Fnc3', 'Fnc4', 'Fnc5', 'Fnc6', 'Fnc7',
    ]
    COLLEN = len(df.columns)

    if COLLEN == 40:
        info += uca
    elif COLLEN == 54:
        info += uca + psd
        PSD = True

    elif COLLEN ==41:
        info += uca + ['EMPTY']

    elif COLLEN ==55:
        info += uca + psd + ['EMPTY']
        PSD = True


    df.rename(inplace=True, columns=lambda x: info[x])
    
    #get the entity name
    ent_name = df.iloc[1,2]
    
    
    # validate the Hourly Rate field ---------------------------------------------------------------------------------------

    #Find the hourly rates, and if it is an " " (whitespace)string, replace it with an empty string ""  
    #Once flagged, we convert it to a 0. 

    # write regex for all non numeric characters. 
    if len(df[df['Hrat'] == ' '].index) > 0 :
    
        print(ent_name + " with "+ batch_name+' has Hrat with \' \' entries. Correcting them and converting to numeric')    
    
        #write to global df
        df_Hrat.loc[len(df_Hrat)] = [ent_name, batch_num, 'entries with \' \'']

        #replace the empty string and convert to numeric.
        df.loc[df.Hrat == ' ', 'Hrat'] = ''
        df['Hrat'] = df['Hrat'].str.replace(',','')
        df['Hrat'] = pd.to_numeric(df['Hrat'],errors='coerce')
    
    else:
        print(batch_name+' Converting to numeric')
        df['Hrat'] = df['Hrat'].str.replace(',','')
        df['Hrat'] = pd.to_numeric(df['Hrat'],errors='coerce')

        #now check that the the Transaction type = PY
  #      if len( df[df['Type'] != 'PY' & df['Hrat'] != 0] ) > 0 : 
#
 #            print(ent_name + " with "+ batch_name+' has Hrat entires without trans Type = PY. Flag and write to error df')    

  #          #write to global df
  #          df_Hrat.loc[len(df_Hrat)] = [ent_name, batch_num, 'transaction type != PY']
  #      else:
        print(batch_name+' has Hrat with no errors. writing info to success df')
        df_valid.loc[len(df_valid)] = [ent_name, batch_num, 'pass']


In [19]:
import glob
batches = glob.glob("H:\\Transparency\\Transparent Utah 3.0\\Raw Batches\\rawBatch\\*.txt")
print(batches[1:5])

['H:\\Transparency\\Transparent Utah 3.0\\Raw Batches\\rawBatch\\2019Q4V2TransparencyExpSLCo.txt', 'H:\\Transparency\\Transparent Utah 3.0\\Raw Batches\\rawBatch\\CPHD 1st Qtr 2020 revised.txt', 'H:\\Transparency\\Transparent Utah 3.0\\Raw Batches\\rawBatch\\FY18 Transparency.txt', 'H:\\Transparency\\Transparent Utah 3.0\\Raw Batches\\rawBatch\\FY20P08_state.txt']


In [29]:
validate_batch("H:/Transparency/Transparent Utah 3.0/Raw Batches/rawBatch/_data_56125.txt")

Mountain Green Sewer Improvement District with batch__data_56125 has Hrat with ' ' entries. Correcting them and converting to numeric


In [30]:
for batch in batches[50:150]:
    #print(batch)
    batch = re.sub("\\\\","/",batch) 
    #print(batch)
    validate_batch(batch)
   

batch__data_58762 Converting to numeric
batch__data_58762 has Hrat with no errors. writing info to success df
batch__data_58821 Converting to numeric
batch__data_58821 has Hrat with no errors. writing info to success df
batch__data_58834 Converting to numeric
batch__data_58834 has Hrat with no errors. writing info to success df
batch__data_58871 Converting to numeric
batch__data_58871 has Hrat with no errors. writing info to success df
batch__data_58876 Converting to numeric
batch__data_58876 has Hrat with no errors. writing info to success df
batch__data_58919 Converting to numeric
batch__data_58919 has Hrat with no errors. writing info to success df
batch__data_58922 Converting to numeric
batch__data_58922 has Hrat with no errors. writing info to success df
batch__data_58938 Converting to numeric
batch__data_58938 has Hrat with no errors. writing info to success df
batch__data_58956 Converting to numeric
batch__data_58956 has Hrat with no errors. writing info to success df
batch__dat

A6 North Davis Prep Academy with batch__data_60476 has Hrat with ' ' entries. Correcting them and converting to numeric
batch__data_60477 Converting to numeric
batch__data_60477 has Hrat with no errors. writing info to success df
1F Quest Academy with batch__data_60478 has Hrat with ' ' entries. Correcting them and converting to numeric
4I Mana Academy  with batch__data_60483 has Hrat with ' ' entries. Correcting them and converting to numeric
batch__data_60492 Converting to numeric
batch__data_60492 has Hrat with no errors. writing info to success df
batch__data_60493 Converting to numeric
batch__data_60493 has Hrat with no errors. writing info to success df
batch__data_60496 Converting to numeric
batch__data_60496 has Hrat with no errors. writing info to success df
batch__data_60498 Converting to numeric
batch__data_60498 has Hrat with no errors. writing info to success df
3L Leadership Academy of Utah with batch__data_60525 has Hrat with ' ' entries. Correcting them and converting t

In [35]:
df_Hrat


,entity_name,batch_num,error_type
0,Mountain Green Sewer Improvement District,_data_56125,entries with ' '
1,Military Installation Development Authority,_data_59617,entries with ' '
2,Moab City,_data_59623,entries with ' '
3,Noth Summit School District,_data_59749,entries with ' '
4,Millville City,_data_60398,entries with ' '
...,...,...,...
57,68 Ogden Prep. Academy,_data_60541,entries with ' '
58,8E Merit Preparatory Academy,_data_60560,entries with ' '
59,Karl G. Maeser Prep Academy,_data_60683,entries with ' '
60,Kanarraville Town,_data_60718,entries with ' '


In [34]:
df_valid.head()

,entity_name,batch_num,error_type
0,PAROWAN CITY,_data_58762,pass
1,TOWN OF JOSEPH,_data_58821,pass
2,CITY OF MILFORD,_data_58834,pass
3,Wellsville-Mendon Conservation District,_data_58871,pass
4,OAKLEY CITY CORPORATION,_data_58876,pass
